In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# pyUsercalc

A jupyter notebook that attempts to recreate most of the functionality of Spiegelman's [UserCalc](http://www.ldeo.columbia.edu/~mspieg/UserCalc) website for calculating Uranium Series disequilibrium calculations for the Equilbrium transport model describe in 

Spiegelman, M., 2000. UserCalc: A Web-based uranium series calculator for magma migration problems. Geochem. Geophys. Geosyst. 1, 1016. https://doi.org/10.1029/1999GC000030


## Some plotting utilities

In [ ]:
import matplotlib as mpl
mpl.rcParams['lines.linewidth']=2
mpl.rcParams['font.size']=16
## Some utility functions for plotting

def plot_inputs(df,figsize=(8,6)):
    ''' 
        pretty plots input data from pandas dataframe df
    '''
    
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, sharey=True,figsize=figsize)
    ax1.plot(df['F'],df['P'])
    ax1.invert_yaxis()
    ax1.set_xlabel('F')
    ax1.set_ylabel('Pressure (kb)')
    xticks = np.linspace(0,max(df['F']),10)
    ax1.grid()
    ax1.set_xticks(xticks,minor=True)
    ax2.plot(df['Kr'],df['P'])
    ax2.set_xlabel('Kr')
    for s in ['DU','DTh','DRa','DPa']:
        ax3.semilogx(df[s],df['P'],label=s)
    ax3.set_xlabel('Ds')
    ax3.legend(loc='best',bbox_to_anchor=(1.1,1))
    
def plot_1Dcolumn(df,figsize=(8,6)):
    '''
        pretty plots output data from dataframe of output
    '''
    
    fig, (ax1, ax2) = plt.subplots(1,2, sharey=True,figsize=figsize)
    ax1.plot(df['phi'],df['P'],'r',label='$\phi$')
    ax1.set_xlabel('Porosity',color='r')
    ax1.set_ylabel('Pressure (kb)')
    ax1.invert_yaxis()

    ax1a = ax1.twiny()
    ax1a.plot(df['F'],df['P'],'b',label='$F$')
    ax1a.set_xlabel('Degree of melting',color='b')

    for s in ['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']:
        ax2.plot(df[s],df['P'],label=s)
    ax2.set_xlabel('Activity Ratios')
    ax2.set_xlim(0,5)
    ax2.set_xticks(range(5))
    ax2.grid()
    ax2.legend(loc='best',bbox_to_anchor=(1.1,1))
    return fig,(ax1,ax1a,ax2)

## A Generic Decay chain solver class

This class solves a generic radiocative decay chain problem for the logarithm of the radiogenic component of  melt concentration (Equation 9) in Spiegelman 2000

$$
    \frac{dU_i^r}{dz} = \lambda'_i\frac{\overline{\rho D_{i}}}{\overline{F D_{i}}}\left[ R_i^{i-1}\exp[U_{i-1}-U_i] - 1\right]
$$

where 
$$
    \lambda'_i = \frac{h\lambda_i}{W_0}
$$ 

are the decay constants scaled by the solid transport time ($h/W_0$) across a layer of depth $h$
$$
    \overline{\rho D_{i}} = \frac{\rho_f}{\rho_s}\phi + (1 -\phi)D_i
$$

is the partition coefficient weighted density (divided by $\rho_s$)
and
$$
    \overline{F D_{i}} = F + (1 -F)D_i
$$


$$
    R_i^{i-1} = \alpha_i\frac{D_i^0}{D_{i-1}^0}\frac{\overline{\rho D_{i-1}}}{\overline{\rho D_{i-1}}}
$$
is the ingrowth factor. $\alpha$ is the inital degree of secular disequilibrium.

Note:  $U_i$ is the total log of the concentration of nuclide $i$ which can be decomposed into
$$
  U_i = U^s_i + U^r_i
$$
where
$$
    U^s_i = \log\left[ \frac{D_i^0}{\overline{FD}_i}\right]
$$
is the log concentration of a stable nuclide with the same partition coefficients.  U^r_i is the radiogenic component.

In [ ]:
from scipy.integrate import solve_ivp

class DecayChain:
    '''
    A class for calculating generic radioactive decay chains for the scaled equations 9 in Spiegelman (2000)
    
    Usage:  solver=DecayChain(alpha0,D0,lambdas,F_bar,rho_bar)
    
    inputs:
        alpha0  :  numpy array of initial activities
        D0      :  numpy array of initial partition coefficients
        lambdas :  decay constants scaled by solid transport time
        F_bar   :  Function that returns F + (1-F)D as a function of z'
        rho_bar :  Function that returns \rhof/rhos\phi + (1-\phi)D as a function of z'
        
    Outputs:  pandas DataFrame with columns z, U, U_s, U_r
    '''
    def __init__(self,alpha0,D0,lambdas,F_bar,rho_bar):
        self.alpha0 = alpha0
        self.D0 = D0
        self.lambdas = lambdas
        self.F_bar = F_bar
        self.rho_bar = rho_bar
        
    def rhs(self,z,Ur):
        '''
        returns right hand side of generic Decay chain problem for the log of the radiogenic concentration
        
        The full equation for dU/dz is given by Eq (9) in Spiegelman 2000 but here we split
        
        U = U^s + U^r where
        U^s is the log of the stable element concentrations U^s = log(D(0)/Fbar_z)
        U^r is the radiogenic ingrowth part
        
        the general equation is 
            dU_i^r/dz = h\lambda_i/Weff_i * [ R_i^{i-1} exp(U_{i-1} - U_i) - 1.)
            
        This routine assumes that lambda, D, D0, lambda_tmp, phi0, W_0 and alpha_0 are set by the driver routine
        '''
        
        # calculate F_bar(z) and rho_bar(z) once
        Fb = self.F_bar(z)
        rb = self.rho_bar(z)
        
        # initial value of partition coefficients
        D0 = self.D0
        # stable concentrations
        Us = np.log(D0/Fb)
        # total concentration
        U = Us + Ur
        # effective velocity and scaled rate term
        lambda_prime = self.lambdas*rb/Fb
        # ingrowth factor and exponential factor
        R = np.zeros(len(lambda_prime))
        expU = np.zeros(len(lambda_prime))
        for i in range(1,len(lambda_prime)):
            R[i] = self.alpha0[i]*D0[i]/D0[i-1]*rb[i-1]/rb[i]
            expU[i] = np.exp(U[i-1]-U[i])
            
        # return full RHS
        
        return lambda_prime*(R*expU - 1.)
        
    def solve(self,z_eval=None):
        '''
        solves generic radioactive decay chain problem as an ODE initial value problem
        if z_eval = None,  save every point
        else save output at every z_eval depth
        '''
        
        # Set initial condition and solve ODE      
        Ur_0 = np.zeros(len(self.D0))
        sol = solve_ivp(self.rhs,(0.,1.),Ur_0,t_eval=z_eval,method='RK45')
        z = sol.t
        Ur = sol.y
        # calculate stable concentration
        Us = np.log(self.D0/self.F_bar(z)).T 
        return z,Ur,Us
        
        

## The main UserCalc class

This class takes in a dataframe of inputs,  initializes the appropriate splines and porosity functions
then presents a set of methods for calculating 1D columns and contour maps

In [ ]:
from scipy.optimize import brentq
from scipy.interpolate import interp1d


class UserCalc:
    ''' A class for constructing solutions for Equilibrium Transport U-series calculations
        ala Spiegelman, 2000, G-cubed
        
        Usage: 
        us = UserCalc(df,dPdz = 0.32373, n = 2., tol=1.e-6, phi0 = 0.01) 
        
        with 
        df   :  a pandas-dataframe with columns ['P','Kr','DU','DTh','DRa','DPa']
        dPdz :  Pressure gradient to convert P to z
        n    :  permeability exponent
        tol  :  tolerance for ODE solver
        phi0 :  initial 
        
        Methods:
            
    '''
    def __init__(self, df, dPdz = 0.32373, n = 2., tol = 1.e-6, phi0 = 0.01, W0 = 3.):
        self.df = df
        self.dPdz = dPdz
        self.n = n
        self.tol = 1.e-6
        self.phi0 = phi0
        self.W0 = W0/1.e5
        
        # set depth scale h
        self.zmin = df['P'].min()/dPdz
        self.zmax = df['P'].max()/dPdz
        self.h = self.zmax - self.zmin
        
        # lambda function to define scaled column height zprime
        self.zp = lambda P: (self.zmax - P/dPdz)/self.h
        
        # set interpolants for F and Kr and pressure
        self.F = interp1d(self.zp(df['P']),df['F'],kind='cubic')
        self.Kr = interp1d(self.zp(df['P']),df['Kr'],kind='cubic')
        self.P = interp1d(self.zp(df['P']),df['P'],kind='cubic')

        # set maximum degree of melting
        self.Fmax = self.df['F'].max()
        
        # set reference densities (assuming a mantle composition)
        self.rho_s = 3300.
        self.rho_f = 2800.
        
        # set  decay constants for [ 238U, 230Th, 226Ra] and [ 235U, 231Pa ]       
        t_half_238 = np.array([4.468e9, 7.54e4, 1600.])
        t_half_235 = np.array([7.03e8, 3.276e4])
        self.lambdas_238 = np.log(2)/t_half_238
        self.lambdas_235 = np.log(2)/t_half_235
        
        # set interpolation functions for Partition coefficients for each chain
        self.D_238 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DTh'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DRa'],kind='cubic') ]
        self.D_235 = [ interp1d(self.zp(df['P']),df['DU'],kind='cubic'),
                       interp1d(self.zp(df['P']),df['DPa'],kind='cubic')]
        
        # lambda function to get partition coefficients at zprime = 0
        self.get_D0 = lambda D: np.array([ D[i](0) for i in range(len(D))])
                        
        # initialize reference permeability
        self.setAd(self.phi0,n=n)
        
        # initialize porosity function
        
    def setAd(self,phi0,n):
        ''' 
            sets the reference permeability given the maximum porosity 
        '''
        Fmax = self.Fmax
        self.phi0 = phi0
        self.Ad =  (self.rho_s/self.rho_f*Fmax - phi0*(1. - Fmax)/(1. - phi0))/(phi0**n*(1.-phi0))
        
    def phi(self,zp):
        '''
        returns porosity as function of dimensionless column height zp
        '''
        # effective permeability
        K = self.Kr(zp)*self.Ad
        
        # degree of melting
        F = self.F(zp)
        
        # density ratio
        rs_rf = self.rho_s/self.rho_f
        
        # rootfinding function to define porosity such that f(phi) = 0
        
        # check if scalar else loop
        if np.isscalar(zp):
             f = lambda phi: K*phi**self.n*(1. - phi)**2 + phi*(1. + F*(rs_rf - 1.)) - F*rs_rf
             phi = brentq(f,0.,1.)
        else: # loop over lenght of zp
            phi = np.zeros(zp.shape)
            for i,z in enumerate(zp):
                f = lambda phi: K[i]*phi**self.n*(1. - phi)**2 + phi*(1. + F[i]*(rs_rf - 1.)) - F[i]*rs_rf
                phi[i] = brentq(f,0.,1.)
        return phi
                
    def F_bar_D(self,zp,D):
        '''
        returns  numpy array of  size (len(zp),len(D)) for
        
        Fbar_D = F + (1. - F)*D_i
        '''
        
        F = self.F(zp)
        if np.isscalar(zp):
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))])
        else :
            F_bar_D = np.zeros((len(zp),len(D)))
            F_bar_D = np.array([ F + (1. - F)*D[i](zp) for i in range(len(D))]).T
        return F_bar_D
    
    def rho_bar_D(self,zp,D):
        '''
        returns  numpy array of  size (len(zp),len(D)) for
        
        rho_bar_D = rho_f/rho_s*phi + (1. - phi)*D_i
        '''
        rho_s = self.rho_s
        rho_f = self.rho_f
        
        phi = self.phi(zp)
        if np.isscalar(zp):
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))])
        else: 
            rho_bar_D = np.zeros((len(zp),len(D)))
            rho_bar_D = np.array([ rho_f/rho_s*phi + (1. - phi)*D[i](zp) for i in range(len(D))]).T
           
        return rho_bar_D
    
    
    def solve_1D(self,phi0 ,n , W0, alphas = np.ones(4), z_eval = 0.):
        '''
        set's up and solves the 1-D column model for a given phi0,n, and upwelling rate W0 (in cm/yr)
        '''
    
        self.setAd(phi0,n)
        self.W0 = W0/self.h
        
        # evaluate at input depths if not specified
        if np.isscalar(z_eval):
            z_eval = self.zp(self.df['P'])
                    
        # solve for the U238 model
        lambdas = self.h*self.lambdas_238/self.W0
        alpha0 = alphas[0:3]
        D0 = self.get_D0(self.D_238)
        F_bar = lambda  z: self.F_bar_D(z,self.D_238)
        rho_bar = lambda z: self.rho_bar_D(z,self.D_238)  
        
        us_238 = DecayChain(alpha0,D0,lambdas,F_bar,rho_bar)
        z,Ur,Us = us_238.solve(z_eval)
        U = Ur + Us
        
        # start building output dataframe
        df = pd.DataFrame()
        df['P'] = self.P(z) 
        df['z'] = self.zmax - self.h*z
        df['F'] = self.F(z)
        df['phi'] = self.phi(z)
        names = ['(230Th/238U)','(226Ra/230Th)']
        for i,name in enumerate(names):
            df[name] = alpha0[i+1]/alpha0[i]*D0[i]/D0[i+1]*np.exp(U[i+1]-U[i])
           
        names = ['U_238U','U_230Th', 'U_226Ra']
        for i,name in enumerate(names):
            df[name] = U[i]
            
        # now solve 235 series
        lambdas = self.h*self.lambdas_235/self.W0
        alpha0 = alphas[[0,3]]
        D0 = self.get_D0(self.D_235)
        F_bar = lambda  z: self.F_bar_D(z,self.D_235)
        rho_bar = lambda z: self.rho_bar_D(z,self.D_235)  
        
        us_235 = DecayChain(alphas,D0,lambdas,F_bar,rho_bar)
        z,Ur,Us = us_235.solve(z_eval)
        U = Ur + Us
        
        df['(231Pa/235U)'] = alpha0[1]/alpha0[0]*D0[0]/D0[1]*np.exp(U[1]-U[0])
        names = ['U_235U','U_231Pa']
        for i,name in enumerate(names):
            df[name] = U[i]
            
        # reorder the data frame
        columns = ['P','z','F','phi','(230Th/238U)','(226Ra/230Th)','(231Pa/235U)',
                                    'U_238U','U_230Th', 'U_226Ra','U_235U','U_231Pa']
        
        
        return df[columns]
        
    
    

## Let's look at some input data

In [ ]:
df = pd.read_csv('data/sample.csv',skiprows=1,dtype=float)

In [ ]:
plot_inputs(df)

### Solve the 1-D column problem

In [ ]:
# initialze the solver object
us = UserCalc(df)

# set column parameters
phi0 = 0.008
W0 = 3. # cm/yr
n = 2.

# and solve at 100 points
z_solve = np.linspace(0,1,100)
df_out = us.solve_1D(phi0,n,W0,z_eval = z_solve)

## activities at the top of the column

In [ ]:
print(df_out[['(230Th/238U)','(226Ra/230Th)','(231Pa/235U)']].iloc[-1])

## Plot solution

In [ ]:
plot_1Dcolumn(df_out)
plt.show()

## view dataframe

In [ ]:
df_out